# Atmosfer pressure. Construction of a regression model.

### Description of data

The system of automatic state ambient air monitoring stations in Lithuania consists of 14 urban air quality monitoring stations operating in Vilnius, Kaunas, Klaipėda, Šiauliai, Panevėžys, Jonava, Kėdainiai, Naujoji Akmenė and Mažeikiai and 3 integrated monitoring stations operating in Aukštaitija, Žemaitija and Dzūkija National Parks.

Concentrations of the following pollutants are measured at automatic air quality monitoring stations: particulate matter PM10, fine particulate matter PM2.5, nitrogen oxides (NO2, NOx, NO), sulfur dioxide (SO2), carbon monoxide (CO), ozone (O3), benzene, mercury.
The tests and measurements shall be carried out in accordance with the requirements of Directives 2004/107/EC of the European Parliament and of the Council relating to arsenic, cadmium, mercury, nickel and polycyclic aromatic hydrocarbons in ambient air and 2008/50/EC on ambient air quality and cleaner air for Europe.


The data consists of files obtained from different sources:

* Averages.csv
* Quantity.csv
* QuantityUnits.csv
* Station.csv

https://data.gov.lt/datasets/500/

## ## Loading the library self-written functions

In [1]:
import pandas as pd
import numpy as np
import time

import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from termcolor import colored

import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots

from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.exceptions import NotFittedError

from sklearn.dummy import DummyRegressor

from lightgbm import LGBMRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
#from catboost import CatBoostRegressor
from xgboost.sklearn import XGBRegressor

warnings.simplefilter(action='ignore', category=FutureWarning)
#from pandas.core.common import SettingWithCopyWarning

%config InlineBackend.figure_format='retina'
#arnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
#warnings.simplefilter(action="ignore", category=UserWarning)

/Users/lesia/anaconda3/lib/python3.11/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction import DictVectorizer
from numpy import asarray

from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [3]:
def bold(): 
    return "\033[1m"

def bold_end(): 
    return "\033[0m"

#Ставим формат для нумериков
pd.options.display.float_format = '{: >10.2f}'.format

In [4]:
#**Function print_basic_info, to display information about the array and its variables.**

#* base - database name
#* info - 1: output information about the array, other: no output
#* describe - 1: output description of array variables, other: no output
#* duplicate - 1: display the number of complete duplicates
#* head - n: output example base (output n - lines), n < 1: no output

def print_basic_info(base, info, describe, duplicat, head):
    if info == 1:
        print("\n", bold(), colored('info','green'), bold_end(), "\n")
        print( base.info())  
    if head >= 1:
        print("\n", bold(),colored('head','green'),bold_end())
        display(base.head(head))
    if describe == 1:
        print("\n", bold(),colored('describe','green'),bold_end(),"\n")
        for i in base.columns:
            print("\n", bold(), colored(i,'blue'),bold_end(),"\n", base[i].describe())
    if duplicat == 1:
        print("\n", bold(),colored('duplicated','green'),bold_end(),"\n")
        print(base[base.duplicated() == True][base.columns[0]].count())

## Loading data

In [ ]:
data_averages = pd.read_csv('datasets/Averages.csv', sep=',',decimal='.')
data_quantity = pd.read_csv('datasets/Quantity.csv', sep=',',decimal='.')
data_quantity_units = pd.read_csv('datasets/QuantityUnits.csv', sep=',',decimal='.')
data_station = pd.read_csv('datasets/Station.csv', sep=',',decimal='.')

<!-- # data_arc = pd.read_csv('/datasets/final_steel/data_arc.csv')
# data_bulk = pd.read_csv('/datasets/final_steel/data_bulk.csv')
# data_bulk_time = pd.read_csv('/datasets/final_steel/data_bulk_time.csv')
# data_gas = pd.read_csv('/datasets/final_steel/data_gas.csv')
# data_temp = pd.read_csv('/datasets/final_steel/data_temp.csv')
# data_wire = pd.read_csv('/datasets/final_steel/data_wire.csv')
# data_wire_time = pd.read_csv('/datasets/final_steel/data_wire_time.csv') -->

## EDA

In [ ]:
print_basic_info(data_averages,1,1,1,5)

In [ ]:
# make data
data_averages['datetime'] = pd.to_datetime(data_averages.ldatetime)

In [ ]:
data_averages['datetime'].hist(bins = 50, alpha=0.5, density=True)

plt.xlabel('dates')
plt.ylabel('Distribution density')
plt.title('datetime distribution by sample', fontsize=15)
plt.show()

2627777 cases of monitoring.

Distribution by time is shows the data is more in 2020 years

In [ ]:
print_basic_info(data_quantity,1,1,1,5)

In [ ]:
data_quantity.longname.value_counts()

90 entries. it is a dictionary what measurements are present 

In [ ]:
print_basic_info(data_quantity_units,1,1,1,5)

962 entries. it is a dictionary about Quantity

In [ ]:
print_basic_info(data_station,1,1,1,5)

In [ ]:
data_station['latitude'].hist(bins = 50, alpha=0.5, density=True)
data_station['longitude'].hist(bins = 50, alpha=0.5, density=True)

plt.title('latitude and longitude', fontsize=15) 
plt.show()

22 entries. it is a dictionary about location (latitude and longitude)

## Data Cleaning

* Where it is necessary to calculate the information by batch and combine the data into one database.
* Leave only the necessary columns, including so that there is no information inaccessible in real cases (leaks).
* We throw it out if the time is the same 

In [ ]:
# combine 
data = data_averages.merge(data_quantity_units,how='left', on = 'code_combi')

# look at target
data.groupby(by="code_unit._id", dropna=False)['lvalue'].mean()

In [ ]:
#data measurements

data['year'] = data.datetime.dt.year
data['month'] = data.datetime.dt.month
data['weekday'] = data.datetime.dt.weekday
data['day'] = data.datetime.dt.day

In [ ]:
# add data_quantity
d_q = data_quantity[data_quantity['code_unit._id'] == '67461e7c-506e-430a-8962-cd25ebed54da'] 
data = data.merge(d_q,how='left', on = 'code_unit._id')

#look for atm.pressure
data_quantity['code_unit._id'].value_counts()

In [ ]:
# leave atm.pressure and 2023 year
data = data[data['year']>= 2023]
data =data[data['code_unit._id'] == '67461e7c-506e-430a-8962-cd25ebed54da']

In [ ]:
# look at target
data.groupby(by="year", dropna=False)['lvalue'].mean()

In [ ]:
# add data_station

data_station['stat_num._id'] = data_station['_id']
data = data.merge(data_station,how='left', on = 'stat_num._id',suffixes=('_xx', '_yy'))
data

In [ ]:
# Distribution Future target
print(bold(),"% mean in test sample: lvalue",colored(round(data['lvalue'].mean(),2),'blue'))

data['lvalue'].hist(bins = 50, alpha=0.5, density=True)

plt.xlabel('lvalue')
plt.ylabel('Distribution density')
plt.title('lvalue distribution by sample', fontsize=15)
plt.show()

In [ ]:
# look at 0.05
data['lvalue'].quantile(0.05)

In [ ]:
# look at 0.99
data['lvalue'].quantile(0.99)

In [ ]:
# del 0,05
data = data[data['lvalue'] > 968]
#data = data[data['lvalue'] < 1025]

In [ ]:
# Distribution Future target
print(bold(),"% mean in test sample: lvalue",colored(round(data['lvalue'].mean(),2),'blue'))

data['lvalue'].hist(bins = 50, alpha=0.5, density=True)

plt.xlabel('lvalue')
plt.ylabel('Distribution density')
plt.title('lvalue distribution by sample', fontsize=15)
plt.show()

In [ ]:
#data = data[['lvalue','stat_num','code_quantity._id','year','month','weekday','day','latitude','longitude']]

# leave our data columns
data = data[['lvalue','stat_num','year','month','weekday','day','latitude','longitude']]

In [ ]:
data[['lvalue','stat_num','year','month','weekday','day','latitude','longitude']].corr()

In [ ]:
#data['stat_num'] = data.stat_num.astype('str')

In [ ]:
print_basic_info(data,1,1,1,5)

In [ ]:
# drop_duplicates

data = data.drop_duplicates()

### Preparing data for the model

* Dividing the date set into test and training/validation.

In [ ]:
#select target
features, target = data.drop(['lvalue'], axis=1), data['lvalue']
features = features.fillna(0)

#we divide into target and test (25%)
features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                                            test_size=0.10, shuffle=False
                                                                           )

In [ ]:
scaler = MinMaxScaler()

scaler.fit(asarray(features_train[['latitude','longitude']]))

features_train[['latitude','longitude']] = scaler.transform(asarray(features_train[['latitude','longitude']]))
features_test[['latitude','longitude']] = scaler.transform(asarray(features_test[['latitude','longitude']]))

In [ ]:
X_train = features_train
X_test = features_test

## Building a model
    
* Identify a pool of approaches that takes into account possible limitations and find a regression model that satisfies the necessary characteristics. 
* Check the model for adequacy.

In [ ]:
#divide into folders
cv = TimeSeriesSplit(n_splits=3)

In [ ]:
#RandomForestClassifier

grid = {'max_depth' : [i for i in np.arange(12,20)]}

clf = RandomForestRegressor()

gs = GridSearchCV(clf, grid, cv=cv, scoring = 'neg_mean_absolute_error')

# temporarily comment out to reduce the calculation time
gs.fit(X_train,target_train)

RandomForestRegressor_params = gs.best_params_
print(RandomForestRegressor_params)

In [ ]:
#XGBRegressor

grid = {'eta' : [i for i in np.arange(0.1,0.3,0.1)] , 'max_depth' : [i for i in np.arange(14,20)]}

clf = XGBRegressor(random_state = 123)

gs = GridSearchCV(clf, grid, cv=cv, scoring = 'neg_mean_absolute_error')

# temporarily comment out to reduce the calculation time
#gs.fit(X_train,target_train)

#XGBRegressor_best_params = gs.best_params_
#print(XGBRegressor_best_params)

In [ ]:
#GradientBoostingRegressor

grid = { 'loss' : ['huber'],
        'learning_rate' : [i for i in np.arange(0.1,0.4,0.1)] , 
        #'max_depth' : [i for i in np.arange(12,17,1)]
       }

clf = GradientBoostingRegressor(random_state = 123)

gs = GridSearchCV(clf, grid, cv=cv, scoring = 'neg_mean_absolute_error')

# temporarily comment out to reduce the calculation time
##gs.fit(X_train,target_train)

#GradientBoostingRegressor_best_params = gs.best_params_
#print(GradientBoostingRegressor_best_params)

In [ ]:
#LGBMRegressor

grid = { 'learning_rate' : [i for i in np.arange(0.1,0.3,0.1)]}

clf = LGBMRegressor(random_state = 123)

gs = GridSearchCV(clf, grid, cv=cv, scoring = 'neg_mean_absolute_error')

# temporarily comment out to reduce the calculation time
#gs.fit(X_train,target_train)

#LGBMRegressor_best_params = gs.best_params_
#print(LGBMRegressor_best_params)

In [ ]:
##let's make a function that will record the training time, prediction speed, and prediction quality
def put_in_base(model_name, base_res, features_train, target_train):
    features_train.reset_index(drop = True, inplace = True)
    target_train.reset_index(drop = True, inplace = True)
    cv = TimeSeriesSplit(n_splits=3)
    time_train, time_predict, MAE = [], [], []
    for train_index, val_index in cv.split(features_train):
        X_train, X_val = features_train.loc[train_index], features_train.loc[val_index]
        y_train, y_val = target_train.loc[train_index], target_train.loc[val_index]
        #training time.
        start_time = time.time()
        model_name.fit(X_train, y_train)
        time_train.append(round((time.time() - start_time),3))
        #prediction speed.
        start_time = time.time()
        predictions_valid = model_name.predict(X_val)
        time_predict.append(round((time.time() - start_time),3))
        #quality of pre-order(MAY)
        MAE.append(mean_absolute_error(y_val, predictions_valid))
    
    if len((str(clf).split('(')[0]).split('.')) == 1:
        base_res.loc[str(clf).split('(')[0],'time_train'] = np.mean(time_train)
        base_res.loc[str(clf).split('(')[0],'time_predict'] = np.mean(time_predict)
        base_res.loc[str(clf).split('(')[0],'MAE'] = np.mean(MAE)
    else:
        nm = ((str(clf).split('(')[0]).split('.')[2]).split(' ')[0]
        base_res.loc[nm,'time_train'] = np.mean(time_train)
        base_res.loc[nm,'time_predict'] = np.mean(time_predict)
        base_res.loc[nm,'MAE'] = np.mean(MAE)
        
    return base_res

In [ ]:
##let's run it for our models with previously found parameters
ans = pd.DataFrame()

for clf in (LinearRegression(),RandomForestRegressor(max_depth = 19),
            XGBRegressor(eta = 0.2, max_depth = 16, random_state = 123),
            GradientBoostingRegressor(learning_rate = 0.4, loss = 'huber', max_depth = 13, random_state = 123),
            LGBMRegressor(learning_rate = 0.2, random_state = 123),
           ):
    put_in_base(clf, ans, X_train,target_train)
    
    
    

In [ ]:
display(ans.sort_values(by = 'MAE'))

## Testing

In [ ]:
final_model = GradientBoostingRegressor(learning_rate = 0.4, loss = 'huber', max_depth = 13, random_state = 123)

final_model.fit(X_train, target_train)

In [ ]:
df_importances = pd.DataFrame()
df_importances['feature'] = data.drop(['lvalue'], axis=1).columns
df_importances['importance'] = final_model.feature_importances_
df_importances

In [ ]:
# del year and stat_num and longitude

X_train = X_train[['month','weekday','day','latitude', 'longitude']]
X_test = X_test[['month','weekday','day','latitude','longitude']]

In [ ]:
def put_in_base_test(model_name, base_res, features_train, target_train, features_test, target_test):
    
    #training time.
    start_time = time.time()
    model_name.fit(features_train, target_train)
    
    if len((str(model_name).split('(')[0]).split('.')) == 1:
        nm = str(model_name).split('(')[0]
    else:
        nm = ((str(model_name).split('(')[0]).split('.')[2]).split(' ')[0]

    base_res.loc[nm,'time_train'] = round((time.time() - start_time),3)
    #prediction speed.
    start_time = time.time()
    predictions_test = model_name.predict(features_test)
    base_res.loc[nm,'time_predict'] = round((time.time() - start_time),3)
    #prediction quality(mae)
    base_res.loc[nm,'MAE'] = mean_absolute_error(target_test, predictions_test)
    
    return predictions_test, model_name, base_res

In [ ]:
# let's look at the result
info_final_test = pd.DataFrame()

predictions_test, final_model, info_final_test = put_in_base_test(final_model, info_final_test, 
                                                                  X_train, target_train, 
                                                                  X_test, target_test)

In [ ]:
info_final_test

In [ ]:
colors = ['rgb(107, 174, 214)']
fig = go.Figure(data=[go.Table( header=dict(
    values=['Время обучения модели (в секундах)', 
            'Время предсказания модели (в секундах)',
            'MAE на тестовой выборке'],
    line_color='white', fill_color='white',
    align='center', font=dict(color='black', size=15)
  ),
  cells=dict(
    values=[info_final_test.time_train.round(2), 
            info_final_test.time_predict.round(2),info_final_test.MAE.round(2)],
    line_color=[colors], fill_color=[colors],
    align='center', font=dict(color='black', size=14)
  ))
])
fig.update_layout(title_text="Показатели финальной модели (GradientBoostingRegressor)", height = 300)
fig.show()

### Checking the model for adequacy.

In [ ]:
print(bold(),"% mean in test sample: target",colored(round(target_test.mean(),2),'blue')+bold(),"VS predictions",
      colored(round(predictions_test.mean() ,2),'blue'),bold_end())

target_test.hist(bins = 50, alpha=0.5, density=True)
pd.Series(predictions_test).hist(bins = 50, alpha=0.5, density=True)

plt.xlabel('Температура нагрева')
plt.ylabel('Плотность распределения')
plt.title('Распределение на тестовой выборке', fontsize=15) 
plt.legend(['Таргет', 'Предсказания'])
plt.show()

The averages are similar. Distributions also, however, the prediction is expected to determine the extreme cases worse

In [ ]:
#Compare dummy
dummy_model = DummyRegressor(strategy='median')

predictions_dummy, dummy_model, info_final_test = put_in_base_test(dummy_model, info_final_test, 
                                                                  features_train, target_train, 
                                                                  features_test, target_test)

In [ ]:
display(info_final_test)

Our model shows itself a little better than Dummy

## Output.
<a name="15."></a>
[<font size="2">(to the content)</font>](#1common.)

* The resulting model has MAY 6.67 on the test sample

Based on the results of the initial analysis and the work taken:
# atm.pressure and 2023 year

***Target attribute - volume (value)***

**The model is built using XGBRegressor(eta = 0.2, max_depth = 16, random_state = 123):**

This model has poor performance comparasing to DummyRegressor and should not be used in the prod